# Introduction

This notebook will contain my work, thought process, and answers to the technical evaluation portion of the take-home project. Instructions have been copied from the Guild SQL Workbench instructions and pasted below.


The Guild Education SQL workbook contains fake data similar to what exists in Guild’s internal database.  Each tab represents one table.  Please make a copy of the workbook, and using the tables provided, write the SQL code necessary to pull the following information from the database (provide 1 query per question and your corresponding answer). Please state any of your assumptions. SQL is required. 


  * Total number of contacts (students) who are not enrolled in any courses. 
  * For each program category count the number of distinct contacts (students) that are enrolled in courses. 
  * Based on your best interpretation of the data, which application type has the most number of successful students? Keep this definition in mind for the python portion below)


### Imports, Connections, and Functions

In [1]:
import pandas as pd
import sqlite3

In [2]:
working_dir = '/Users/jmbeck/Desktop/guild_eval'

In [3]:
conn = sqlite3.connect(working_dir + '/db/eval_db.sqlite')

In [4]:
# This returns query rows as a dictionary, and allows referencing results by variable name. 
conn.row_factory = sqlite3.Row

In [5]:
cu = conn.cursor()

## Work

The answers to my work are included below.  For the sake of clarity I have not included my EDA on the structure of the database in this answers notebook. Rather, those insights were incldued in the 00_create_sqlite_db notebook.

One important note is that I have changed table names from the worksheet to not have spaces in them, so 'CONTACT DETAILS' is now stored as CONTACT_DETAILS. 

### Question 1

**Total number of contacts (students) who are not enrolled in any courses.**

For this question, I will use the CONTACT_DETAILS table in my sqlite db.  I will make a couple assumptions using the data. 

  1. A contact will be defined by their unique SF Contact ID.
  2. Course enrollment will be determined using the presence of a SF Course C ID in the CONTACT_DETAILS table.
 
To get number of courses people are enrolled in a course, I will leverage the fact that using COUNT in a specific field ignores NULL values in SQL.  Therefore, we can run the entire query using just the CONTACT_DETAILS table.

In [6]:
students_not_enrolled_in_courses = '''
SELECT COUNT([Sf Contact Id]) AS student_count
FROM (
--Inline view of grouped data for enrolled courses per contact
SELECT [Sf Contact Id], COUNT([Sf Course C ID]) AS COURSES_ENROLLED
FROM CONTACT_DETAILS
GROUP BY [Sf Contact ID]
HAVING COUNT([Sf Course C ID]) = 0
)
'''

In [7]:
_ = cu.execute(students_not_enrolled_in_courses)

In [8]:
result = cu.fetchone()

In [9]:
print(result['student_count'])

16336


There are 16336 students in the database who are not enrolled in any courses.

### Question 2

**For each program category count the number of distinct contacts (students) that are enrolled in courses.**

Assumptions:
  1. A contact can be enrolled in multiple programs - identifiable by the Opportunity id. This fact was observed in the database creation script where I ran basic aggregations on each ID across tables.

Two tables will need to be used in this query, the CONTACT_DETAILS, and OPPORTUNITY_DETAILS table.  I will need to join the CONTACT_DETAILS table and the OPPORTUNITY_DETAILS using the [Sf Opportunity ID] field. Filtering down to just the enrolled students will be handled by reversing the HAVING logic from the original query, and using a Common Table Expression to simplify the structure of the query.

In [10]:
enrolled_students_with_category_query = '''
-- Leverage a CTE for enrolled students query to simplify
WITH enrolled_students AS
(SELECT [Sf Contact Id], [Sf Opportunity Id], COUNT([Sf Course C ID]) AS COURSES_ENROLLED
FROM CONTACT_DETAILS
GROUP BY [Sf Contact ID], [Sf Opportunity Id]
HAVING COUNT([Sf Course C ID]) > 1)

SELECT [Sf Opportunity Program Category], COUNT(DISTINCT [Sf Contact Id]) AS n_enrollees
FROM enrolled_students A
LEFT JOIN OPPORTUNITY_DETAILS B
ON A.[Sf Opportunity Id] = B.[Sf Opportunity Id]
GROUP BY [Sf Opportunity Program Category]
ORDER BY COUNT(DISTINCT [Sf Contact Id]) DESC
'''

The output will be pulled and displayed in pandas for clarity.

In [11]:
enrolled_students_with_category = pd.read_sql(enrolled_students_with_category_query, conn)

In [12]:
enrolled_students_with_category

,Sf Opportunity Program Category,n_enrollees
0,University,704
1,Lead Gen - Secondary,167
2,Credential,2
3,StraighterLine/Saylor,2
4,ELL Program,1
5,Lead Gen - Post Secondary,1


### Question 3

**Based on your best interpretation of the data, which application type has the most number of successful students**

I'll leverage all three tables for this query. The first step will be to identify successful course completions in the COURSE_DETAILS table. The logic is explained in the assumptions below. Successful course completions can be linked to the CONTACT_DETAILS table to get the contact_id associated with a successful course completion, and the opportunity_id to join to the OPPORTUNITY_DETAILS table. 

Assumptions:
  * The list of successful grades will include ('A','A-','B+','B','B-','C+','C','C-','D+','D','D-','Passed','S')
  * A student will be considered successful if they have been successful in any course. 
  * With some additional information this query could easily be expanded to calculate a GPA for each student based on their grades, and that overall GPA could be used to determine 'success'
  * Since the query asks for the largest number of students, I will return that, but since application types have highly imbalanced counts, an alternative way to evaluate success would be the fraction of students who are successful. 
  * I am counting unique students, not unique courses taken, so the final query will Count the distinct Sf Contact IDs. 

Definitions of the letter grades and codes here:  
  * https://www.york.cuny.edu/academics/policies/grading-policies 
  * https://catalog.csun.edu/policies/administrative-grading-symbols-i-ic-rp-sp-w-wu-cr-and-nc/

In [13]:
successful_course_query = '''
WITH successful_student_course AS
(
SELECT A.[Sf Contact ID]
 , A.[Sf Opportunity ID]
 , B.*
FROM CONTACT_DETAILS A
LEFT JOIN COURSE_DETAILS B
ON A.[Sf Course C ID] = B.[Sf Course C ID]
WHERE [Sf Course C Final Grade C] in ('A','A-','B+','B','B-','C+','C','C-','D+','D','D-','Passed','S')
)

SELECT [Sf Opportunity Application Type C] AS Application_Type
  , COUNT(DISTINCT [Sf Contact ID]) AS successful_students
FROM (
SELECT A.*
 , B.[Sf Opportunity Application Type C]
FROM successful_student_course A
LEFT JOIN OPPORTUNITY_DETAILS B
ON A.[Sf Opportunity ID] = B.[Sf Opportunity ID] ) D
GROUP BY [Sf Opportunity Application Type C]
ORDER BY COUNT(DISTINCT [Sf Contact ID]) DESC
LIMIT 1
'''

In [14]:
_ = cu.execute(successful_course_query)

In [15]:
q3_result = cu.fetchone()

In [16]:
print('The top application type is', q3_result['Application_Type'],'with', q3_result['successful_students'], 'successful students')

The top application type is Guild Education with 486 successful students
